-----------------------
#### prepare the data for the KS/KG
--------------------------

In [2]:
import pandas as pd
import csv
import ast 

In [3]:
# Load the CSV file
df = pd.read_csv("exported_data.csv")

In [4]:
df.sample(3)

,_id,_labels,born,lastSeen,name,released,tagline,taglineEmbedding,title,_start,_end,_type,rating,roles,summary
324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,116.0,ACTED_IN,NaN,"[""Evey Hammond""]",NaN
61,61.0,:Person,1956.0,NaN,Vincent Ward,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0,87.0,DIRECTED,NaN,NaN,NaN


In [5]:
# Separate nodes (where _start and _end are NaN or empty)
nodes_df = df[df['_start'].isnull()]

In [6]:
# Separate relationships (where _start and _end are not NaN)
relationships_df = df.dropna(subset=['_start', '_end'])

In [7]:
nodes_df.shape, relationships_df.shape

((172, 15), (253, 15))

In [8]:
nodes_df.sample(3)

,_id,_labels,born,lastSeen,name,released,tagline,taglineEmbedding,title,_start,_end,_type,rating,roles,summary
109,109.0,:Person,1969.0,NaN,David Mitchell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,161.0,:Movie,NaN,NaN,NaN,2004.0,This Holiday Season... Believe,"[-6.733057E-4,-0.017166452,-0.013592816,-0.017...",The Polar Express,NaN,NaN,NaN,NaN,NaN,NaN
33,33.0,:Person,1933.0,NaN,Tom Skerritt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
relationships_df.sample(3)

,_id,_labels,born,lastSeen,name,released,tagline,taglineEmbedding,title,_start,_end,_type,rating,roles,summary
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,56.0,ACTED_IN,NaN,"[""The Tracker""]",NaN
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,52.0,DIRECTED,NaN,NaN,NaN
399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158.0,157.0,DIRECTED,NaN,NaN,NaN


In [10]:
# Keep only relevant columns for nodes (no 'rating')
nodes_df = nodes_df[['_id', '_labels', 'born', 'lastSeen', 'name', 'released', 'tagline', 'taglineEmbedding', 'title']]

# Keep the relevant columns for relationships (including 'rating')
relationships_df = relationships_df[['_id', '_start', '_end', '_type', 'rating', 'roles', 'summary']]

In [14]:
nodes_df.sample(3)

,_id,_labels,born,lastSeen,name,released,tagline,taglineEmbedding,title
140,140.0,:Person,1960.0,NaN,Oliver Platt,NaN,NaN,NaN,NaN
168,168.0,:Person,NaN,NaN,Angela Scope,NaN,NaN,NaN,NaN
159,159.0,:Movie,NaN,NaN,NaN,2007.0,A stiff drink. A little mascara. A lot of nerv...,"[-0.0074488586,-0.014382637,-8.118299E-4,-0.01...",Charlie Wilson's War


In [15]:
relationships_df.sample(3)

,_id,_start,_end,_type,rating,roles,summary
356,NaN,132.0,130.0,ACTED_IN,NaN,"[""Brutus \""Brutal\"" Howell""]",NaN
186,NaN,7.0,9.0,PRODUCED,NaN,NaN,NaN
259,NaN,68.0,67.0,ACTED_IN,NaN,"[""Patricia Eden""]",NaN


In [23]:
# Save them to separate files if needed
nodes_df.to_csv('nodes_data.csv', index=False)
relationships_df.to_csv('relationships_data.csv', index=False)

#### load data

In [24]:
from neo4j import GraphDatabase

In [25]:
# neo4j - Aura DB free (cloud)
NEO4J_URI     = 'neo4j+s://264d8780.databases.neo4j.io'
NEO4J_USERNAME= 'neo4j'
NEO4J_PASSWORD= '5l2648jhBn6kzOFi_XcK_yzYCVFzZIpoOPW7xp7M_Ss'
NEO4J_DATABASE= 'neo4j'
AURA_INSTANCEID= '264d8780'
AURA_INSTANCENAME = 'Instance01'

In [26]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [27]:
def delete_all_nodes_and_relationships():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    with driver.session() as session:
        session.run(query)
    print("All nodes and relationships deleted.")

# Call the function
delete_all_nodes_and_relationships()

All nodes and relationships deleted.


**nodes - movies**

In [28]:
def load_nodes(csv_file, query):
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Process only rows where _labels is ':Movie'
            if row['_labels'] == ':Movie':
                # Parse taglineEmbedding from string to list of floats if available
                tagline_embedding = ast.literal_eval(row['taglineEmbedding']) if row.get('taglineEmbedding') else None
                
                # Extract other values from the row, handle any missing values (None)
                params = {key.strip(): value.strip() if value else None for key, value in row.items()}
                params['taglineEmbedding'] = tagline_embedding  # Add taglineEmbedding to params

                # Rename _id to id in the params dictionary for Neo4j
                params['id'] = params.pop('_id')  # This ensures the key for the Cypher query is 'id'

                # Debugging: Print the parameters to ensure the correct 'id' field
                #print(f"Params for Movie: {params}")

                # Execute the query with parameters
                with driver.session() as session:
                    session.run(query, params)

In [29]:
# Cypher query for nodes (adjust based on your CSV structure)
node_query = """
MERGE (n:Movie {id: $id})
SET n.name = $name,
    n.released         = $released,
    n.tagline          = $tagline,
    n.title            = $title,
    n.taglineEmbedding = $taglineEmbedding
"""

In [30]:
# Load nodes and relationships
load_nodes('nodes_data.csv', node_query)

**Query the movie nodes**

In [31]:
# Function to query Movie nodes
def query_movies():
    with driver.session() as session:
        
        result = session.run("MATCH (m:Movie) RETURN m.id AS id, m.title AS title, m.released AS released, m.tagline AS tagline LIMIT 20")
        
        for record in result:
            print(f"Movie ID: {record['id']}, Title: {record['title']}, Released: {record['released']}, Tagline: {record['tagline']}")

In [32]:
query_movies()

Movie ID: 0.0, Title: The Matrix, Released: 1999.0, Tagline: Welcome to the Real World
Movie ID: 9.0, Title: The Matrix Reloaded, Released: 2003.0, Tagline: Free your mind
Movie ID: 10.0, Title: The Matrix Revolutions, Released: 2003.0, Tagline: Everything that has a beginning has an end
Movie ID: 11.0, Title: The Devil's Advocate, Released: 1997.0, Tagline: Evil has its winning ways
Movie ID: 15.0, Title: A Few Good Men, Released: 1992.0, Tagline: In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.
Movie ID: 29.0, Title: Top Gun, Released: 1986.0, Tagline: I feel the need, the need for speed.
Movie ID: 37.0, Title: Jerry Maguire, Released: 2000.0, Tagline: The rest of his life begins now.
Movie ID: 46.0, Title: Stand By Me, Released: 1986.0, Tagline: For some, it's the last real taste of innocence, and the first real taste of life. But for everyone, it's the time t

**add Nodes - person**

In [33]:
def load_person_nodes(csv_file):
    person_query = """
    MERGE (p:Person {id: $id})
    SET p.name = $name,
        p.born = $born,
        p.lastSeen = $lastSeen
    """
    
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['_labels'] == ':Person':
                # Prepare parameters
                params = {
                    'id': row['_id'],
                    'name': row.get('name'),
                    # Convert values to integers if they exist
                    'born': int(float(row['born'])) if row.get('born') else None,
                    'lastSeen': int(float(row['lastSeen'])) if row.get('lastSeen') else None,
                }
                # Execute query
                with driver.session() as session:
                    session.run(person_query, params)

    print("Person nodes loaded successfully.")

In [34]:
person_query = """
MERGE (p:Person {id: $id})
SET p.name = $name,
    p.born = $born,
    p.lastSeen = $lastSeen
"""

In [35]:
load_person_nodes('nodes_data.csv')

Person nodes loaded successfully.


**query the persons nodes**

In [36]:
def query_person_nodes():
    person_query = """
    MATCH (p:Person)
    RETURN p.id AS id, p.name AS name, p.born AS born, p.lastSeen AS lastSeen
    LIMIT 10
    """
    
    with driver.session() as session:
        results = session.run(person_query)
        for record in results:
            print(f"ID: {record['id']}, Name: {record['name']}, Born: {record['born']}, Last Seen: {record['lastSeen']}")

In [37]:
query_person_nodes()

ID: 1.0, Name: Keanu Reeves, Born: 1964, Last Seen: None
ID: 2.0, Name: Carrie-Anne Moss, Born: 1967, Last Seen: None
ID: 3.0, Name: Laurence Fishburne, Born: 1961, Last Seen: None
ID: 4.0, Name: Hugo Weaving, Born: 1960, Last Seen: None
ID: 5.0, Name: Lilly Wachowski, Born: 1967, Last Seen: None
ID: 6.0, Name: Lana Wachowski, Born: 1965, Last Seen: None
ID: 7.0, Name: Joel Silver, Born: 1952, Last Seen: None
ID: 8.0, Name: Emil Eifrem, Born: 1978, Last Seen: None
ID: 12.0, Name: Charlize Theron, Born: 1975, Last Seen: None
ID: 13.0, Name: Al Pacino, Born: 1940, Last Seen: None


#### load relations

In [38]:
relationships_df.sample(5)

,_id,_start,_end,_type,rating,roles,summary
308,NaN,104.0,100.0,DIRECTED,NaN,NaN,NaN
387,NaN,151.0,150.0,DIRECTED,NaN,NaN,NaN
242,NaN,54.0,52.0,ACTED_IN,NaN,"[""Simon Bishop""]",NaN
224,NaN,40.0,37.0,ACTED_IN,NaN,"[""Frank Cushman""]",NaN
400,NaN,71.0,159.0,ACTED_IN,NaN,"[""Rep. Charlie Wilson""]",NaN


In [39]:
# Function to load relationships
def load_relationships(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        with driver.session() as session:
            for row in reader:
                # Extract relationship details
                start_id = row['_start']
                end_id = row['_end']
                rel_type = row['_type']
                
                # Prepare additional properties (if any)
                properties = {key: row[key] for key in row if key not in ['_start', '_end', '_type'] and row[key]}

                # Build the Cypher query dynamically
                query = f"""
                MATCH (start), (end)
                WHERE start.id = $start_id AND end.id = $end_id
                CREATE (start)-[r:{rel_type} {{ {', '.join(f'{k}: ${k}' for k in properties)} }}]->(end)
                """
                
                # Combine parameters
                params = {'start_id': start_id, 'end_id': end_id}
                params.update(properties)

                # Run the query
                session.run(query, params)

In [40]:
load_relationships('relationships_data.csv')

#### some more queries

In [41]:
# Function to query all movies
def query_movies():
    with driver.session(database=NEO4J_DATABASE) as session:
        query = """
        MATCH (m:Movie)
        RETURN m.id AS MovieID, m.title AS Title, m.released AS Released, m.tagline AS Tagline
        LIMIT 10
        """
        result = session.run(query)
        for record in result:
            print(f"Movie ID: {record['MovieID']}, Name: {record['Title']}, Released: {record['Released']}, Tagline: {record['Tagline']}")

In [42]:
query_movies()

Movie ID: 0.0, Name: The Matrix, Released: 1999.0, Tagline: Welcome to the Real World
Movie ID: 9.0, Name: The Matrix Reloaded, Released: 2003.0, Tagline: Free your mind
Movie ID: 10.0, Name: The Matrix Revolutions, Released: 2003.0, Tagline: Everything that has a beginning has an end
Movie ID: 11.0, Name: The Devil's Advocate, Released: 1997.0, Tagline: Evil has its winning ways
Movie ID: 15.0, Name: A Few Good Men, Released: 1992.0, Tagline: In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.
Movie ID: 29.0, Name: Top Gun, Released: 1986.0, Tagline: I feel the need, the need for speed.
Movie ID: 37.0, Name: Jerry Maguire, Released: 2000.0, Tagline: The rest of his life begins now.
Movie ID: 46.0, Name: Stand By Me, Released: 1986.0, Tagline: For some, it's the last real taste of innocence, and the first real taste of life. But for everyone, it's the time that memo

In [44]:
# Function to query all persons
def query_persons():
    with driver.session(database=NEO4J_DATABASE) as session:
        query = """
        MATCH (p:Person)
        RETURN p.id AS PersonID, p.name AS Name, p.born AS Born, p.lastSeen AS LastSeen
        LIMIT 10
        """
        result = session.run(query)
        for record in result:
            print(f"Person ID: {record['PersonID']}, Name: {record['Name']}, Born: {record['Born']}, Last Seen: {record['LastSeen']}")

In [45]:
query_persons()

Person ID: 1.0, Name: Keanu Reeves, Born: 1964, Last Seen: None
Person ID: 2.0, Name: Carrie-Anne Moss, Born: 1967, Last Seen: None
Person ID: 3.0, Name: Laurence Fishburne, Born: 1961, Last Seen: None
Person ID: 4.0, Name: Hugo Weaving, Born: 1960, Last Seen: None
Person ID: 5.0, Name: Lilly Wachowski, Born: 1967, Last Seen: None
Person ID: 6.0, Name: Lana Wachowski, Born: 1965, Last Seen: None
Person ID: 7.0, Name: Joel Silver, Born: 1952, Last Seen: None
Person ID: 8.0, Name: Emil Eifrem, Born: 1978, Last Seen: None
Person ID: 12.0, Name: Charlize Theron, Born: 1975, Last Seen: None
Person ID: 13.0, Name: Al Pacino, Born: 1940, Last Seen: None


In [46]:
# Function to find all movies a person acted in
def query_person_movies(person_name):
    with driver.session(database=NEO4J_DATABASE) as session:
        query = """
        MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
        WHERE p.name = $person_name
        RETURN p.name AS Actor, m.title AS MovieName, r.roles AS Roles
        """
        result = session.run(query, person_name=person_name)
        for record in result:
            print(f"Actor: {record['Actor']}, Movie: {record['MovieName']}, Roles: {record['Roles']}")

In [47]:
query_person_movies("Tom Hanks")

Actor: Tom Hanks, Movie: You've Got Mail, Roles: ["Joe Fox"]
Actor: Tom Hanks, Movie: Sleepless in Seattle, Roles: ["Sam Baldwin"]
Actor: Tom Hanks, Movie: Joe Versus the Volcano, Roles: ["Joe Banks"]
Actor: Tom Hanks, Movie: That Thing You Do, Roles: ["Mr. White"]
Actor: Tom Hanks, Movie: Cloud Atlas, Roles: ["Zachry","Dr. Henry Goose","Isaac Sachs","Dermot Hoggins"]
Actor: Tom Hanks, Movie: The Da Vinci Code, Roles: ["Dr. Robert Langdon"]
Actor: Tom Hanks, Movie: The Green Mile, Roles: ["Paul Edgecomb"]
Actor: Tom Hanks, Movie: Apollo 13, Roles: ["Jim Lovell"]
Actor: Tom Hanks, Movie: Cast Away, Roles: ["Chuck Noland"]
Actor: Tom Hanks, Movie: Charlie Wilson's War, Roles: ["Rep. Charlie Wilson"]
Actor: Tom Hanks, Movie: The Polar Express, Roles: ["Hero Boy","Father","Conductor","Hobo","Scrooge","Santa Claus"]
Actor: Tom Hanks, Movie: A League of Their Own, Roles: ["Jimmy Dugan"]


In [50]:
# Function to find all actors in a specific movie
def query_movie_actors(movie_name):
    with driver.session(database=NEO4J_DATABASE) as session:
        query = """
        MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
        WHERE m.title = $movie_name
        RETURN m.name AS Movie, p.name AS Actor, r.roles AS Roles
        """
        result = session.run(query, movie_name=movie_name)
        for record in result:
            print(f"Movie: {record['Movie']}, Actor: {record['Actor']}, Roles: {record['Roles']}")

In [51]:
query_movie_actors("Cloud Atlas")

Movie: None, Actor: Hugo Weaving, Roles: ["Bill Smoke","Haskell Moore","Tadeusz Kesselring","Nurse Noakes","Boardman Mephi","Old Georgie"]
Movie: None, Actor: Tom Hanks, Roles: ["Zachry","Dr. Henry Goose","Isaac Sachs","Dermot Hoggins"]
Movie: None, Actor: Halle Berry, Roles: ["Luisa Rey","Jocasta Ayrs","Ovid","Meronym"]
Movie: None, Actor: Jim Broadbent, Roles: ["Vyvyan Ayrs","Captain Molyneux","Timothy Cavendish"]
